# TensorFlow BYOM: Train locally and deploy on SageMaker.


1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
    1. [Permissions and environment variables](#Permissions-and-environment-variables)
    2. [Model definitions](#Model-definitions)
    3. [Data Setup](#Data-setup)
3. [Training the network locally](#Training)
4. [Set up hosting for the model](#Set-up-hosting-for-the-model)
    1. [Export from TensorFlow](#Export-the-model-from-tensorflow)
    2. [Import model into SageMaker](#Import-model-into-SageMaker)
    3. [Create endpoint](#Create-endpoint) 
5. [Validate the endpoint for use](#Validate-the-endpoint-for-use)

__Note__: Compare this with the [tensorflow bring your own model example](../tensorflow_iris_byom/tensorflow_BYOM_iris.ipynb)

This notebook was last tested on a ml.m5.xlarge instance running the Python 3 (Data Science image) kernel in SageMaker Studio.

## Introduction 

We will do a classification task, training locally in the box from where this notebook is being run. We then set up a real-time hosted endpoint in SageMaker.

Consider the following model definition for IRIS classification. This mode uses the ``tensorflow.estimator.DNNClassifier`` which is a pre-defined estimator module for its model definition.

## Prequisites and Preprocessing
### Permissions and environment variables

Here we set up the linkage and authentication to AWS services. In this notebook we only need the roles used to give learning and hosting access to your data. The Sagemaker SDK will use S3 defualt buckets when needed. If the ``get_execution_role``  does not return a role with the appropriate permissions, you'll need to specify an IAM role arn that does.

In [2]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 43.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 8.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 41.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 44.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.1/178.1 kB 4.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import boto3
import numpy as np
import os
import pandas as pd
import re
import sagemaker
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.utils import S3DataConfig

import shutil
import tarfile
import tensorflow as tf
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Input, Dense

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sm_session = sagemaker.Session()
bucket_name = sm_session.default_bucket()

### Model Definitions

For this example, we'll use a very simple network architecture, with three densely-connected layers.

In [4]:
def iris_mlp(metrics):
    ### Setup loss and output node activation
    output_activation = "softmax"
    loss = "sparse_categorical_crossentropy"

    input = Input(shape=(4,), name="input")

    x = Dense(
        units=10,
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        activation="relu",
        name="dense_layer1",
    )(input)

    x = Dense(
        units=20,
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        activation="relu",
        name="dense_layer2",
    )(x)

    x = Dense(
        units=10,
        activation="relu",
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        name="dense_layer3",
    )(x)

    output = Dense(units=3, activation=output_activation)(x)

    ### Compile the model
    model = tf.keras.Model(input, output)

    model.compile(optimizer="adam", loss=loss, metrics=metrics)

    return model

### Data Setup

We'll use the pre-processed iris training and test data stored in a public S3 bucket for this example.

In [13]:
# S3 bucket where the original data is downloaded and stored.
downloaded_data_bucket = f"jumpstart-cache-prod-{region}"
downloaded_data_prefix = "training-datasets/tf_byom"

def check_bucket_permission(bucket):
    # check if the bucket exists
    permission = False
    try:
        boto3.Session().client("s3").head_bucket(Bucket=bucket)
    except botocore.exceptions.ParamValidationError as e:
        print(
            "Hey! You either forgot to specify your S3 bucket"
            " or you gave your bucket an invalid name!"
        )
    except botocore.exceptions.ClientError as e:
        if e.response["Error"]["Code"] == "403":
            print(f"Hey! You don't have permission to access the bucket, {bucket}.")
        elif e.response["Error"]["Code"] == "404":
            print(f"Hey! Your bucket, {bucket}, doesn't exist!")
        else:
            raise
    else:
        permission = True
    return permission

if check_bucket_permission(downloaded_data_bucket):
    print(f"Downloaded training data will be read from s3://{downloaded_data_bucket}/{downloaded_data_prefix}")

Downloaded training data will be read from s3://jumpstart-cache-prod-us-east-1/training-datasets/tf_byom


In [14]:
s3 = boto3.client("s3")

filename = "iris_train.csv"
s3.download_file(downloaded_data_bucket, f"{downloaded_data_prefix}/{filename}", filename)

filename = "iris_test.csv"
s3.download_file(downloaded_data_bucket, f"{downloaded_data_prefix}/{filename}", filename)

filename = "iris.data"
s3.download_file(downloaded_data_bucket, f"{downloaded_data_prefix}/{filename}", filename)

In [15]:
# Load the training and test data from .csv to a Pandas data frame.
train_df = pd.read_csv(
    "iris_train.csv",
    header=0,
    names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"],
)
test_df = pd.read_csv(
    "iris_test.csv",
    header=0,
    names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"],
)

# Pop the record labels into N x 1 Numpy arrays
train_labels = np.array(train_df.pop("class"))
test_labels = np.array(test_df.pop("class"))

# Save the remaining features as Numpy arrays
train_np = np.array(train_df)
test_np = np.array(test_df)

## Training the Network Locally

Here, we train the network using the Tensorflow .fit method, just like if we were using our local computers. This should only take a few seconds because the model is so simple.

In [16]:
EPOCHS = 50
BATCH_SIZE = 32

EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", mode="auto", restore_best_weights=True
)

# Instantiate classifier
classifier = iris_mlp(metrics=["accuracy", "binary_accuracy"])

# Fit classifier
history = classifier.fit(
    x=train_np,
    y=train_labels,
    validation_data=(test_np, test_labels),
    callbacks=[EARLY_STOPPING],
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
)

Epoch 1/50
4/4 [==============================] - 1s 76ms/step - loss: 1.4402 - accuracy: 0.3000 - binary_accuracy: 0.4472 - val_loss: 1.1113 - val_accuracy: 0.4667 - val_binary_accuracy: 0.4000
Epoch 2/50
4/4 [==============================] - 0s 14ms/step - loss: 1.2396 - accuracy: 0.3000 - binary_accuracy: 0.4194 - val_loss: 1.0415 - val_accuracy: 0.4000 - val_binary_accuracy: 0.3111


## Set up hosting for the model

### Export the model from tensorflow

In order to set up hosting, we have to import the model from training to hosting. We will begin by exporting the model from TensorFlow and saving it to our file system. We also need to convert the model into a form that is readable by ``sagemaker.tensorflow.model.TensorFlowModel``. There is a small difference between a SageMaker model and a TensorFlow model. The conversion is easy and fairly trivial. Simply move the tensorflow exported model into a directory ``export\Servo\`` and tar the entire directory. SageMaker will recognize this as a loadable TensorFlow model.

In [17]:
classifier.save("export/Servo/1")
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("export")

INFO:tensorflow:Assets written to: export/Servo/1/assets


INFO:tensorflow:Assets written to: export/Servo/1/assets


Open a new sagemaker session and upload the model on to the default S3 bucket. We can use the ``sagemaker.Session.upload_data`` method to do this. We need the location of where we exported the model from TensorFlow and where in our default bucket we want to store the model(``/model``). The default S3 bucket can be found using the ``sagemaker.Session.default_bucket`` method.

Here, we upload the model to S3

In [18]:
s3_response = sm_session.upload_data("model.tar.gz", bucket=bucket_name, key_prefix="model")

### Import model into SageMaker

Use the ``sagemaker.tensorflow.model.TensorFlowModel`` to import the model into SageMaker that can be deployed. We need the location of the S3 bucket where we have the model and the role for authentication.

In [19]:
sagemaker_model = TensorFlowModel(
    model_data=f"s3://{bucket_name}/model/model.tar.gz",
    role=role,
    framework_version="2.3",
)

### Create endpoint

Now the model is ready to be deployed at a SageMaker endpoint. We can use the ``sagemaker.tensorflow.model.TensorFlowModel.deploy`` method to do this. Unless you have created or prefer other instances, we recommend using a single ``'ml.m5.2xlarge'`` instance for this example. These are supplied as arguments. 

In [20]:
%%time

import time
from time import strftime, gmtime

endpoint_name = "jumpstart-example-infer-tf-byom-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)

predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type="ml.m5.2xlarge", endpoint_name=endpoint_name)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


jumpstart-example-infer-tf-byom-2023-04-05-02-39-29
---!CPU times: user 394 ms, sys: 17.3 ms, total: 411 ms
Wall time: 2min 2s


### Validate the endpoint for use

We can now use this endpoint to classify an example to ensure that it works. The output from `predict` will be an array of probabilities for each of the 3 classes.

In [21]:
sample = [6.4, 3.2, 4.5, 1.5]
predictor.predict(sample)

{'predictions': [[0.170156583, 0.491346359, 0.338497072]]}

Delete all temporary directories so that we are not affecting the next run. Also, optionally delete the end points.

In [22]:
os.remove("model.tar.gz")
os.remove("iris_test.csv")
os.remove("iris_train.csv")
os.remove("iris.data")
shutil.rmtree("export")

If you do not want to continue using the endpoint, you can remove it. Remember, open endpoints are charged. If this is a simple test or practice, it is recommended to delete them.

In [23]:
predictor.delete_endpoint()